# TorchArrow in 10 minutes

TorchArrow is a torch.Tensor-like Python DataFrame library for data preprocessing in deep learning. It supports multiple execution runtimes and Arrow as a common memory format.

(Remark. In case the following looks familiar, it is with gratitude that portions of this tutorial were borrowed and adapted from the 10 Minutes to Pandas (and CuDF) tutorial.)



The TorchArrow library consists of 3 parts: 

  * *DTypes* define *Schema*, *Fields*, primitive and composite *Types*. 
  * *Columns* defines sequences of strongly typed data with vectorized operations.
  * *Dataframes*  are sequences of named and typed columns of same length with relational operations.  

Let's get started...

In [1]:
import torcharrow as ta
import torcharrow.dtypes as dt


## Constructing data: Columns

### From Pandas to TorchArrow
To start let's create a Panda series and a TorchArrow column and compare them:

In [2]:
import pandas as pd


pd.Series([1, 2, None, 4])


0    1.0
1    2.0
2    NaN
3    4.0
dtype: float64

In Pandas each Series has an index, here depicted as the first column. Note also that the inferred type is float and not int, since in Pandas None implicitly  promotes an int list to a float series.

TorchArrow has a much more precise type system:

In [3]:
s = ta.column([1, 2, None, 4])
s


0  1
1  2
2  None
3  4
dtype: Int64(nullable=True), length: 4, null_count: 1

TorchArrow creates CPU column by default, which is supported by [Velox](https://github.com/facebookincubator/velox) backend.

In [4]:
s.device


'cpu'

TorchArrow infers that the type is `Int64(nullable=True)`. Of course, we can always get lots of more information from a column: the length, null_count determine the total number, and the number of nulls, respectively.




In [5]:
len(s), s.null_count


(4, 1)

TorchArrow infers Python float as float32 (instead of float64). This follows PyTorch and other deep learning libraries. 

In [6]:
ss = ta.column([2.718, 3.14, 42.42])
ss


0   2.718
1   3.14
2  42.42
dtype: float32, length: 3, null_count: 0

TorchArrow supports (almost all of Arrow types), including arbitrarily nested structs, maps, lists, and fixed size lists. Here is a non-nullable column of a list of non-nullable strings of arbitrary length.

In [7]:
sf = ta.column([["hello", "world"], ["how", "are", "you"]], dtype=dt.List(dt.string))
sf.dtype


List(item_dtype=String(nullable=False), nullable=False, fixed_size=-1)

And here is a column of average climate data, one map per continent, with city as key and yearly average min and max temperature:


In [8]:
mf = ta.column(
    [
        {"helsinki": [-1.3, 21.5], "moscow": [-4.0, 24.3]},
        {"algiers": [11.2, 25.2], "kinshasa": [22.2, 26.8]},
    ]
)
mf


0  {'helsinki': [-1.2999999523162842, 21.5], 'moscow': [-4.0, 24.299999237060547]}
1  {'algiers': [11.199999809265137, 25.200000762939453], 'kinshasa': [22.200000762939453, 26.799999237060547]}
dtype: Map(string, List(float32)), length: 2, null_count: 0

### Append and concat

Columns are immutable. Use `append` to create a new column with a list of values appended. 

In [9]:
sf = sf.append([["I", "am", "fine", "and", "you"]])
sf


0  ['hello', 'world']
1  ['how', 'are', 'you']
2  ['I', 'am', 'fine', 'and', 'you']
dtype: List(string), length: 3, null_count: 0

Use `torcharrow.concat` to combine a list of columns.

In [10]:
sf = ta.concat([sf, ta.column([["I", "am", "fine", "too"]])])
sf

0  ['hello', 'world']
1  ['how', 'are', 'you']
2  ['I', 'am', 'fine', 'and', 'you']
3  ['I', 'am', 'fine', 'too']
dtype: List(string), length: 4, null_count: 0

## Constructing data: Dataframes

A Dataframe is just a set of named and strongly typed columns of equal length:

In [11]:
df = ta.dataframe(
    {"a": list(range(7)), "b": list(reversed(range(7))), "c": list(range(7))}
)
df


  index    a    b    c
-------  ---  ---  ---
      0    0    6    0
      1    1    5    1
      2    2    4    2
      3    3    3    3
      4    4    2    4
      5    5    1    5
      6    6    0    6
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64)]), count: 7, null_count: 0

To access a dataframes columns write:

In [12]:
df.columns


['a', 'b', 'c']

Dataframes are also immutable, except you can always add a new column or overwrite an existing column. 

When a new column is added, it is appended to the set of existing columns at the end.

In [13]:
df["d"] = ta.column(list(range(99, 99 + 7)))
df


  index    a    b    c    d
-------  ---  ---  ---  ---
      0    0    6    0   99
      1    1    5    1  100
      2    2    4    2  101
      3    3    3    3  102
      4    4    2    4  103
      5    5    1    5  104
      6    6    0    6  105
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 7, null_count: 0

You can also overwrite an existing column.

In [14]:
df["c"] = df["b"] * 2
df


  index    a    b    c    d
-------  ---  ---  ---  ---
      0    0    6   12   99
      1    1    5   10  100
      2    2    4    8  101
      3    3    3    6  102
      4    4    2    4  103
      5    5    1    2  104
      6    6    0    0  105
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 7, null_count: 0

Similar to Column, we can also use `append` to create a new DataFrame with a listed of tuples appended.

In [15]:
df = df.append([(7, 77, 777, 7777), (8, 88, 888, 8888)])
df


  index    a    b    c     d
-------  ---  ---  ---  ----
      0    0    6   12    99
      1    1    5   10   100
      2    2    4    8   101
      3    3    3    6   102
      4    4    2    4   103
      5    5    1    2   104
      6    6    0    0   105
      7    7   77  777  7777
      8    8   88  888  8888
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 9, null_count: 0

Dataframes can be nested. Here is a Dataframe having sub-dataframes. 


In [16]:
df_inner = ta.dataframe({"b1": [11, 22, 33], "b2": [111, 222, 333]})
df_outer = ta.dataframe({"a": [1, 2, 3], "b": df_inner})
df_outer


  index    a  b
-------  ---  ---------
      0    1  (11, 111)
      1    2  (22, 222)
      2    3  (33, 333)
dtype: Struct([Field('a', int64), Field('b', Struct([Field('b1', int64), Field('b2', int64)]))]), count: 3, null_count: 0

## Interop

Coming soon!

## Viewing (sorted) data

Take the the top n rows

In [17]:
df.head(2)


  index    a    b    c    d
-------  ---  ---  ---  ---
      0    0    6   12   99
      1    1    5   10  100
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 2, null_count: 0

Or return the last n rows

In [18]:
df.tail(1)


  index    a    b    c     d
-------  ---  ---  ---  ----
      0    8   88  888  8888
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 1, null_count: 0

or sort the values before hand.

In [19]:
df.sort(by=["c", "b"]).head(2)


  index    a    b    c    d
-------  ---  ---  ---  ---
      0    6    0    0  105
      1    5    1    2  104
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 2, null_count: 0

Sorting can be controlled not only by which columns to sort on, but also whether to sort ascending or descending, and how to deal with nulls, are they listed first or last.  

## Selection using Indices

Torcharrow supports two indices:
 - Integer indices select rows
 - String indices select columns

So projecting a single column of a dataframe is simply

In [20]:
df['a']

0  0
1  1
2  2
3  3
4  4
5  5
6  6
7  7
8  8
dtype: int64, length: 9, null_count: 0

Selecting a single row uses an integer index. (In TorchArrow everything is zero-based.)

In [21]:
df[1]

(1, 5, 10, 100)

Selecting a slice keeps the type alive. Here we slice rows:


In [22]:
df[2:6:2]

  index    a    b    c    d
-------  ---  ---  ---  ---
      0    2    4    8  101
      1    4    2    4  103
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 2, null_count: 0

TorchArrow follows the normal Python semantics for slices: that is a slice interval is closed on the left and open on the right.

## Selection by Condition

Selection of a column or dataframe *c* by a condition takes a boolean column *b* of the same length as *c*. If the *i*th row in *b* is true, *c*'s *i*th row is included in the result otherwise it is dropped. Below expression selects the first row, since it is true, and drops all remaining rows, since they are false.



In [23]:
df[[True] + [False] * (len(df)-1)]

  index    a    b    c    d
-------  ---  ---  ---  ---
      0    0    6   12   99
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 1, null_count: 0

Conditional expressions over vectors return boolean vectors. Conditionals are thus the usual way to write filters. 

In [24]:
b = df['a'] > 4
df[b]

  index    a    b    c     d
-------  ---  ---  ---  ----
      0    5    1    2   104
      1    6    0    0   105
      2    7   77  777  7777
      3    8   88  888  8888
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 4, null_count: 0

Torcharrow supports all the usual predicates, like ==,!=,<,<=,>,>= as well as _in_. The later is denoted by `isin`


In [25]:
df[df['a'].isin([5])]

  index    a    b    c    d
-------  ---  ---  ---  ---
      0    5    1    2  104
dtype: Struct([Field('a', int64), Field('b', int64), Field('c', int64), Field('d', int64)]), count: 1, null_count: 0

## Missing data
 Missing data can be filled in via the `fill_null` method 

In [26]:
t = s.fill_null(999)
t

0    1
1    2
2  999
3    4
dtype: int64, length: 4, null_count: 0

Alternatively data that has null data can be dropped:

In [27]:
s.drop_null()

0  1
1  2
2  4
dtype: Int64(nullable=True), length: 3, null_count: 0

## Operators
Columns and dataframes support all of Python's usual binary operators, like  ==,!=,<,<=,>,>= for equality  and comparison,  +,-,*,**,/,// for performing arithmetic and &,|,~ for conjunction, disjunction and negation. 

The semantics of each operator is given by lifting their scalar operation to vectors and dataframes. So given for instance a scalar comparison operator, in TorchArrow a scalar can be compared to each item in a column, two columns can be compared pointwise, a column can be compared to each column of a dataframe, and two dataframes can be compared by comparing each of their respective columns. 

Here are some example expressions:

In [28]:
u = ta.column(list(range(5)))
v = -u
w = v+1
v*w

0   0
1   0
2   2
3   6
4  12
dtype: int64, length: 5, null_count: 0

In [29]:
uv = ta.dataframe({"a": u, "b": v})
uu = ta.dataframe({"a": u, "b": u})
(uv == uu)


  index  a     b
-------  ----  -----
      0  True  True
      1  True  False
      2  True  False
      3  True  False
      4  True  False
dtype: Struct([Field('a', boolean), Field('b', boolean)]), count: 5, null_count: 0

## Null strictness

The default behavior of torcharrow operators and functions is that *if any argument is null then the result is null*. For instance:

In [30]:
u = ta.column([1, None, 3])
v = ta.column([11, None, None])
u + v


0  12
1  None
2  None
dtype: Int64(nullable=True), length: 3, null_count: 2

If null strictness does not work for your code you could call first `fill_null` to provide a value that is used instead of null. 

## Numerical columns and descriptive statistics
Numerical columns also support lifted operations, for `abs`, `ceil`, `floor`, `round`. Even more excited might be to use their aggregation operators like `count`, `sum`, `prod`, `min`, `max`, or descriptive statistics like `std`, `mean`, `median`, and `mode`. Here is an example ensemble:


In [31]:
(t.min(), t.max(), t.sum(), t.mean())


(1, 999, 1006, 251.5)

The `describe` method puts this nicely together: 

In [32]:
t.describe()


  index  statistic      value
-------  -----------  -------
      0  count          4
      1  mean         251.5
      2  std          498.335
      3  min            1
      4  25%            1.75
      5  50%            3
      6  75%          252.75
      7  max          999
dtype: Struct([Field('statistic', string), Field('value', float64)]), count: 8, null_count: 0

Sum, prod, min and max are also available as accumulating operators called `cumsum`, `cumprod`, etc. 

Boolean vectors are very similar to numerical vector. They offer the aggregation operators `any` and `all`. 

## String, list and map methods
Torcharrow provides all of Python's string, list and map processing methods, just lifted to work over columns. Like in Pandas they are all accessible via the `str`, `list` and `map` property, respectively.

### Strings
Let's convert a Column of strings to upper case. 


In [33]:
s = ta.column(["what a wonderful world!", "really?"])
s.str.upper()


0  'WHAT A WONDERFUL WORLD!'
1  'REALLY?'
dtype: string, length: 2, null_count: 0, device: cpu

We can also split each string to a list of strings with the given delimiter.


In [34]:
ss = s.str.split(pat=" ")
ss


0  ['what', 'a', 'wonderful', 'world!']
1  ['really?']
dtype: List(String(nullable=True)), length: 2, null_count: 0

### Lists

To operate on a list column use the usual pure list operations, like `len(gth)`, `slice`, `index` and `count`, etc. But there are a couple of additional operations. 

For instance to invert the result of a string split operation a list of string column also offers a join operation. 


In [35]:
ss.list.join(sep="-")


0  'what-a-wonderful-world!'
1  'really?'
dtype: String(nullable=True), length: 2, null_count: 0, device: cpu

In addition lists provide `filter`, `map`, `flatmap` and `reduce` operators, which we will discuss as in more details in functional tools.

### Maps

Column of type map provide the usual map operations like `len(gth)`, `[]`, `keys` and `values`. Keys and values both return a list column. Key and value columns can be reassembled by calling `mapsto`.

In [36]:
mf.maps.keys()


0  ['helsinki', 'moscow']
1  ['algiers', 'kinshasa']
dtype: List(String(nullable=True)), length: 2, null_count: 0

## Relational tools: Where, select, groupby, join, etc.
 
TorchArrow also plans to support all relational operators on DataFrame. The following sections discuss what exists today.

### Where
The simplest operator is `df.where(p)` which is just another way of writing `df[p]`. (Note: TorchArrow's `where`  is not Pandas' `where`, the latter is a vectorized if-then-else which we call in Torcharrow `ite`.)

In [37]:
xf = ta.dataframe({"A": ["a", "b", "a", "b"], "B": [1, 2, 3, 4], "C": [10, 11, 12, 13]})

xf.where(xf["B"] > 2)


  index  A      B    C
-------  ---  ---  ---
      0  a      3   12
      1  b      4   13
dtype: Struct([Field('A', string), Field('B', int64), Field('C', int64)]), count: 2, null_count: 0

Note that in `xf.where` the predicate `xf['B']>2` refers to self, i.e. `xf`. To access self in an expression TorchArrow introduces the special name `me`. That is, we can also write:


In [38]:
from torcharrow import me


xf.where(me["B"] > 2)


  index  A      B    C
-------  ---  ---  ---
      0  a      3   12
      1  b      4   13
dtype: Struct([Field('A', string), Field('B', int64), Field('C', int64)]), count: 2, null_count: 0

### Select

Select is SQL's standard way to define a new set of columns. We use *positional args to keep columns and kwargs to give new bindings. Here is a typical example that keeps all of xf's columns but adds column 'D').


In [39]:
xf.select(*xf.columns, D=me["B"] + me["C"])


  index  A      B    C    D
-------  ---  ---  ---  ---
      0  a      1   10   11
      1  b      2   11   13
      2  a      3   12   15
      3  b      4   13   17
dtype: Struct([Field('A', string), Field('B', int64), Field('C', int64), Field('D', int64)]), count: 4, null_count: 0

The short form of `*xf.columns` is '\*', so `xf.select('*', D=me['B']+me['C'])` does the same.

### Grouping, Join and Tranpose

Coming soon!


## Functional tools:  map, filter, reduce

Column and dataframe pipelines support map/reduce style programming as well. We first explore column oriented operations.

###  Map and its variations

`map` maps values of a column according to input correspondence. The input correspondence can be given as a mapping or as a (user-defined-) function (UDF). If the mapping is a dict, then non mapped values become null.




In [40]:
ta.column([1, 2, None, 4]).map({1: 111})


0  111
1  None
2  None
3  None
dtype: Int64(nullable=True), length: 4, null_count: 3

If the mapping is a defaultdict, all values will be mapped as described by the default dict.

In [41]:
from collections import defaultdict
ta.column([1, 2, None, 4]).map(defaultdict(lambda: -1, {1: 111}))


0  111
1   -1
2   -1
3   -1
dtype: Int64(nullable=True), length: 4, null_count: 0

**Handling null.** If the mapping is a function, then it will be applied on all values (including null), unless na_action is `'ignore'`, in which case, null values are passed through.

In [42]:
def add_ten(num):
    return num + 10

ta.column([1, 2, None, 4]).map(add_ten, na_action="ignore")


0  11
1  12
2  None
3  14
dtype: Int64(nullable=True), length: 4, null_count: 1

Note that `.map(add_ten, na_action=None)` would fail with a type error since `add_ten` is not defined for `None`/null. So if we wanted to pass null to `add_ten` we would have to prepare for it, maybe like so:

In [43]:
def add_ten_or_0(num):
    return 0 if num is None else num + 10


ta.column([1, 2, None, 4]).map(add_ten_or_0, na_action=None)


0  11
1  12
2   0
3  14
dtype: Int64(nullable=True), length: 4, null_count: 0

**Mapping to different types.** If `map` returns a column type that is different from the input column type, then `map` has to specify the returned column type. 

In [44]:
ta.column([1,2,3,4]).map(str, dtype=dt.string)

0  '1'
1  '2'
2  '3'
3  '4'
dtype: string, length: 4, null_count: 0, device: cpu

Instead of specifying `dtype` argument, you can also rely on type annotations (both Python annotations and `dtypes` are supported):

In [45]:
from typing import Optional


def str_only_even(x) -> Optional[str]:
    if x % 2 == 0:
        return str(x)
    return None


ta.column([1, 2, 3, 4]).map(str_only_even)  # dt.string(nullable=True) is inferred


0  None
1  '2'
2  None
3  '4'
dtype: String(nullable=True), length: 4, null_count: 2, device: cpu

**Map over Dataframes** Of course, `map` works over Dataframes, too. In this case the callable gets the whole row as a tuple. 

In [46]:
def add_unary(tup):
    return tup[0] + tup[1]


ta.dataframe({"a": [1, 2, 3], "b": [1, 2, 3]}).map(add_unary, dtype=dt.int64)


0  2
1  4
2  6
dtype: int64, length: 3, null_count: 0

**Multi-parameter functions**. So far all our functions were unary functions. But `map` can be used for n-ary functions, too: simply specify the set of `columns` you want to pass to the nary function. 


In [47]:
def add_binary(a, b):
    return a + b


ta.dataframe({"a": [1, 2, 3], "b": ["a", "b", "c"], "c": [1, 2, 3]}).map(
    add_binary, columns=["a", "c"], dtype=dt.int64
)


0  2
1  4
2  6
dtype: int64, length: 3, null_count: 0

**Multi-return functions.** Functions that return more than one column can be specified by returning a dataframe  (aka as struct column); providing the  return type is mandatory.

In [48]:
ta.dataframe({"a": [17, 29, 30], "b": [3, 5, 11]}).map(
    divmod,
    columns=["a", "b"],
    dtype=dt.Struct([dt.Field("quotient", dt.int64), dt.Field("remainder", dt.int64)]),
)


  index    quotient    remainder
-------  ----------  -----------
      0           5            2
      1           5            4
      2           2            8
dtype: Struct([Field('quotient', int64), Field('remainder', int64)]), count: 3, null_count: 0

**Functions with state**. Functions sometimes need additional precomputed state. We capture the state in a (data)class and use a method as a delegate:
 

In [49]:
def fib(n):
    if n == 0:
        return 0
    elif n == 1 or n == 2:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)


from dataclasses import dataclass


@dataclass
class State:
    state: int

    def __post_init__(self):
        self.state = fib(self.state)

    def add_fib(self, x):
        return self.state + x


m = State(10)
ta.column([1, 2, 3]).map(m.add_fib)


0  56
1  57
2  58
dtype: int64, length: 3, null_count: 0

TorchArrow requires that only global functions or methods on class instances can be used as user defined functions. Lambdas, which can can capture arbitrary state and are not inspectable, are not supported. 

### Filter

`filter` takes a predicate and returns all those rows for which the predicate holds: 

In [50]:
ta.column([1, 2, 3, 4]).filter(lambda x: x % 2 == 1)


0  1
1  3
dtype: int64, length: 2, null_count: 0

Instead of the predicate you can pass an iterable of boolean of the same length as the column: 

In [51]:
ta.column([1, 2, 3, 4]).filter([True, False, True, False])


0  1
1  3
dtype: int64, length: 2, null_count: 0

If the predicate is an n-ary function, use the  `columns` argument as we have seen for `map`.  

### Flatmap

`flatmap` combines `map` with `filter`. Each callable can return a list of elements. If that list is empty, flatmap filters, if the returned list is a singleton, flatmap acts like map, if it returns several elements it 'explodes' the input. Here is an example: 

In [52]:
def selfish(words):
    return [words, words] if len(words) >= 1 and words[0] == "I" else []


sf.flatmap(selfish)


0  ['I', 'am', 'fine', 'and', 'you']
1  ['I', 'am', 'fine', 'and', 'you']
2  ['I', 'am', 'fine', 'too']
3  ['I', 'am', 'fine', 'too']
dtype: List(string), length: 4, null_count: 0

`flatmap` has all the flexibility of `map`, i.e it can take the `ignore`, `dtype` and `column` arguments.

### Reduce
`reduce` is just like Python's `reduce`. Here we compute the product of a column.

In [53]:
import operator


ta.column([1, 2, 3, 4]).reduce(operator.mul)


24

## Batch Transform


Batch `transform` is simliar to `map`, except the functions takes batch input/output (represented by a Python list, PyTorch tensors, etc).

In [54]:
from typing import List


def multiple_ten(val: List[int]) -> List[int]:
    return [x * 10 for x in val]


ta.column([1, 2, 3, 4]).transform(multiple_ten, format="python")


0  10
1  20
2  30
3  40
dtype: int64, length: 4, null_count: 0

# (Beta) torcharrow.functional

Velox UDFs can be registered in TorchArrow, which is available in `torcharrow.functional`. 
Especially all the [Velox core SQL functions](https://facebookincubator.github.io/velox/functions.html) are 
already included. 

Here is an example usage of Velox string function 
[lpad](https://facebookincubator.github.io/velox/functions/string.html#lpad):


In [55]:
from torcharrow import functional

col = ta.column(["abc", "x", "yz"])
# Velox's lpad function: https://facebookincubator.github.io/velox/functions/string.html#lpad
functional.lpad(col, 5, "123")

0  '12abc'
1  '1231x'
2  '123yz'
dtype: String(nullable=True), length: 3, null_count: 0, device: cpu

Here is another example usage of Velox array function 
[array_except](https://facebookincubator.github.io/velox/functions/array.html#array_except): 

In [56]:
col1 = ta.column([[1, 2, 3], [1, 2, 3], [1, 2, 2], [1, 2, 2]])
col2 = ta.column([[4, 5, 6], [1, 2], [1, 1, 2], [1, 3, 4]])

# Velox's array_except function: https://facebookincubator.github.io/velox/functions/array.html#array_except
functional.array_except(col1, col2)

0  [1, 2, 3]
1  [3]
2  []
3  [2]
dtype: List(Int64(nullable=True), nullable=True), length: 4, null_count: 0

You can also implement your own transformation functions and 
register it, so it becomes avaiable in `torcharrow.functional`. 

Guide about how to implement your own Velox transformation and register
it in TorchArrow is coming soon!

For example the `bucketize` function used in recommendation systems, 
as implemented [here](https://github.com/facebookresearch/torcharrow/blob/1abdc0e7fa25480c0c9a301b05b854f93254c0cc/csrc/velox/functions/rec/bucketize.h). 



In [57]:
col = ta.column([1, 2, 3, 5, 8, 10, 11])
buckets = [2, 5, 10]
functional.bucketize(col, buckets)

0  0
1  0
2  1
3  1
4  2
5  2
6  3
dtype: Int32(nullable=True), length: 7, null_count: 0

In [58]:
"End of tutorial"


'End of tutorial'